### --- Day 24: Blizzard Basin ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/24

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 24);
WriteLines(inputLines);

Loading puzzle file: Day24.txt
Total lines: 37
Max line length: 102

#.####################################################################################################
#>>^<^v>v<>^>>v<<.<<<^<vv^<^>v><^<v>><^>.<><^^>v<^>v>^^>v><v.^<.^>>>^>v^^^<vv^<><vvv>v>^vv<<>v^^<v><>#
#>^>^><^v<>.v><><<<>><v><>vv^.><v^<^v^>v<<<v..<v.v^v.v<<v<>>^^<v^v.<^.^v><^v^.^^<v>..v<^>^<<^>.v.>v.>#
#.>^>v<v^><v<v^><>^v>><>v<>^><>v^<^v<>v<>^<v<><^^v.v.>^^.v<<^v<>>.v>>v<^^.v<v^<^>>..>><<v.^>.<<<^v.^>#
#<>^^^..v^^v^<<><<<^<>.>.vvv.v>^^.><.^>>>v^^>^^vv>v<^v>>^^<v>v^<<^>..^v^<>>>>>.>>><v^^^vv<>v^v<>^^.<>#


In [4]:
string[] testInputLines = 
[
   "#.######",
   "#>>.<^<#",
   "#.<..<<#",
   "#>v.><>#",
   "#<^v^^>#",
   "######.#",
];

Wow, that is a lot of blizzards! Almost every square has a blizzard on it. Tracking and updating the positions of the blizzards every minute looks expensive. Having said that, after some further thought, the blizzards travel in one direction and do not interfere with each other, so their paths will loop predictably. All the horizontal bilizzards will loop back to their origin after $width$ minutes, and all the vertical blizzards will loop after $height$ minutes, and the entire blizzard state will loop modulo the lowest common multiple of $width$ x $height$. For the main puzzle input the size is 35 x 100, so our cycle time is 700 minutes. That is 20 loops horizontally and 7 vertically.

Therefore it is possible to build a map of free / occupied spaces for every minute of the cycle, and thus every minute thereafter, and we can use that to search our way to the goal. But this is not a simple shortest path problem as we may need to stand in place to avoid a blizzard, or even dodge one by stepping forward and then back again!

I think we'll need to try depth-first search with pruning like we did for Day 19. I think there's a few ways we can reduce the search space:

1. Manhattan distance pruning: if the current point at the current minute can't possibly reach the goal faster than our current time even in the best-case scenario, we can skip.
2. "Equivalence" of positions: we know that the blizzard states loop, so if we have reached a given position a full cycle earlier on a previous search, there's no point searching it again. This is especially important to prevent the search from perpetually standing in place or stepping back / forth between two points.
3. I suspect it's "better" to prioritise steps that take us closer to the goal. I.e., step down rather than up. Hopefully we can get an early baseline that helps prune the space quickly.

Let's see how it goes...

In [5]:
using PointTime = (Point point, int time);
const char Wall = '#';

int FindGoalTime(string[] inputLines)
{
    CharGrid grid = new(inputLines);
    var rows = inputLines.Length - 2;
    var cols = inputLines[0].Length - 2;

    Console.WriteLine($"Grid space is {rows} x {cols}");
    var cycleMinutes = LCM(rows, cols);
    Console.WriteLine($"Cycle repeats every {cycleMinutes} mins");

    var startPoint = grid.Enumerate().Where(pch => pch.point.Y is 0 && pch.ch is not Wall).Single().point;
    var bottomRow = inputLines.Length - 1;
    var goalPoint = grid.Enumerate().Where(pch => pch.point.Y == bottomRow && pch.ch is not Wall).Single().point;

    HashSet<char> blizzardChars = [ '<', '>', '^', 'v' ];
    var allBlizzards = grid.Enumerate().Where(gch => blizzardChars.Contains(gch.ch)).ToList();

    Dictionary<char, Point> blizDirs = new()
    {
        { '<', Left },
        { '>', Right },
        { 'v', Down },
        { '^', Up }
    };

    // Build the map of free / occupied spaces for every minute of the cycle
    var allPoints = Range(1, rows).SelectMany(y => Range(1, cols).Select(x => new Point(x, y)));
    var freePoints = allPoints.ToDictionary(
        ap => ap,
        ap => Range(0, cycleMinutes).Select(_ => true).ToList()
    );
    foreach (var blizz in allBlizzards)
    {
        var blizDir = blizDirs[blizz.ch];
        var curr = blizz.point;
        foreach (var i in Range(0, cycleMinutes))
        {
            freePoints[curr][i] = false;
            curr = Move(curr, blizDir);
        }
    }

    // The points in time we've seen, so we can ignore equivalents (item 2
    // above)
    HashSet<PointTime> equivalents = new();

    PointTime searchStart = (startPoint, 0);
    var currentBest = int.MaxValue;
    
    var discoveredPoints = DFS(searchStart, x => {
        if (HasEquivalent(x))
        {
            return [];
        }
        equivalents.Add(x);

        var xBest = x.time + Distance(x.point, goalPoint);
        if (xBest > currentBest)
        {
            return [];
        }

        if (x.point == startPoint)
        {
            return GetFirstSteps(x);
        }

        if (x.point == goalPoint)
        {
            return [];
        }

        return GetNextPoints(x);
    });

    foreach (var (point, time) in discoveredPoints)
    {
        if (point == goalPoint)
        {
            if (time < currentBest)
            {
                // We found a new best!
                currentBest = time;
            }
        }
    }

    return currentBest;

    // Helper functions:
    /////

    // Move the blizzards and wrap them if they hit the wall
    Point Move(Point p, Point dir)
    {
        p += dir;
        if (grid[p] is Wall)
        {
            p += dir;
            var newX = p.X switch {
                // Off the left
                -1 => cols,
                // Off the right
                var x when x == cols + 2 => 1,
                // Inside map
                _ => p.X
            };
            var newY = p.Y switch {
                // Off the top
                -1 => rows,
                // Off the bottom
                var y when y == rows + 2 => 1,
                // Inside map
                _ => p.Y
            };
            p = (newX, newY);
        }

        return p;
    }

    // Have we seen this point in time earlier?
    bool HasEquivalent(PointTime maybeNext) => Equivalents(maybeNext).Any(m => equivalents.Contains(m));

    // All the ealier equivalent points in time
    IEnumerable<PointTime> Equivalents(PointTime check)
    {
        var x = check.time / cycleMinutes;
        var m = check.time % cycleMinutes;
        return Enumerable.Range(0, x).Select(t => m + t * cycleMinutes).Select(t => (check.point, t));
    }

    // Manhattan distance
    int Distance(Point from, Point to)
    {
        var xDist = Math.Abs(to.X - from.X);
        var yDist = Math.Abs(to.Y - from.Y);
        return xDist + yDist;
    }

    // 
    IEnumerable<PointTime> GetNextPoints(PointTime pt)
    {
        var (point, time) = pt;
        var allChecks = Dirs4.Select(dir => point + dir).Append(point)
            .Select(p => (point: p, dist: Distance(p, goalPoint)));

        allChecks = allChecks.OrderBy(x => x.dist);
        var tNext = time + 1;
        foreach (var (maybePoint, dist) in allChecks)
        {
            // Start points will be found by GetFirstSteps
            if (maybePoint == startPoint) { continue; }

            // If we hit the end from here, no point finding other paths that
            // take longer time
            if (maybePoint == goalPoint) 
            {
                yield return (maybePoint, tNext);
                yield break;
            }
             
            if (grid[maybePoint] is Wall) { continue; }

            // Prune if we cannot improve
            if (tNext + dist >= currentBest) { continue; }

            // Is this space free from blizzards in the next minute?
            if (!freePoints[maybePoint][tNext % cycleMinutes]) { continue; }

            PointTime maybeNext = (maybePoint, tNext);
            
            // Have we reached this point in an earlier cycle?
            if (HasEquivalent(maybeNext)) { continue; }

            yield return maybeNext;
        }
    }

    IEnumerable<PointTime> GetFirstSteps(PointTime origin)
    {
        var (point, time) = origin;
        if (time == cycleMinutes - 1)
        {
            // We've seen all these combinations already
            yield break;
        }

        var firstStep = point + Down;
        var tNext = time + 1;
        if (freePoints[firstStep][tNext])
        {
            // Take the step down
            yield return (firstStep, tNext);
        }
        // Wait in place for another minute
        yield return (point, tNext);
    }
}

In [6]:
string[] meTest = [
    "#.####",
    "#..<.#",
    "#.v..#",
    "####.#",
];
var meTestAnswer = FindGoalTime(meTest);
Console.WriteLine($"Minimum goal time: {meTestAnswer} minutes");

Grid space is 2 x 4
Cycle repeats every 4 mins
Minimum goal time: 6 minutes


In [7]:
// In the above example, the fastest way to reach your goal requires 18 steps.

var testAnswer = FindGoalTime(testInputLines);
Console.WriteLine(testAnswer);

Grid space is 4 x 6
Cycle repeats every 12 mins
18


In [8]:
// What is the fewest number of minutes required to avoid the blizzards and reach the goal?

var part1Answer = FindGoalTime(inputLines);
Console.WriteLine(part1Answer);

Grid space is 35 x 100
Cycle repeats every 700 mins
228


In [9]:
// 228 is correct!
Ensure(228, part1Answer);